# Groupby

The groupby method allows you to group rows of data together and call aggregate functions

In [189]:
import numpy as np
import pandas as pd
# Create dataframe
data = {'Company':['GOOG','MSFT','FB','MSFT','GOOG','FB',],
       'Person':['Sam','Charlie','Amy','Vanessa','Carl','Sarah'],
       'Sales':[200,120,340,124,243,350]}

In [190]:
df = pd.DataFrame(data)

In [191]:
df

,Company,Person,Sales
0,GOOG,Sam,200
1,MSFT,Charlie,120
2,FB,Amy,340
3,MSFT,Vanessa,124
4,GOOG,Carl,243
5,FB,Sarah,350


** Now you can use the .groupby() method to group rows together based off of a column name. For instance let's group based off of Company. This will create a DataFrameGroupBy object:**

In [197]:
by_comp = df.groupby("Company")

In [198]:
type(by_comp)

pandas.core.groupby.generic.DataFrameGroupBy

The DataFrameGroupby is like a dict or table where the keys are the modalitie of 'Company' column and the values are frames containing rows corresponding to this modality

In [199]:
for company, frame in by_comp:
    print(f"First 2 entries for {company!r}")
    print("------------------------")
    print(frame.head(2), end="\n\n")


First 2 entries for 'FB'
------------------------
  Company Person  Sales
2      FB    Amy    340
5      FB  Sarah    350

First 2 entries for 'GOOG'
------------------------
  Company Person  Sales
0    GOOG    Sam    200
4    GOOG   Carl    243

First 2 entries for 'MSFT'
------------------------
  Company   Person  Sales
1    MSFT  Charlie    120
3    MSFT  Vanessa    124



One term that’s frequently used alongside .groupby() is split-apply-combine. This refers to a chain of three steps:

    - Split a table into groups
    - Apply some operations to each of those smaller tables
    - Combine the results

In [207]:
by_comp.get_group('FB')

,Company,Person,Sales
2,FB,Amy,340
5,FB,Sarah,350


You can display the indices

In [208]:
#indices of groups
by_comp.indices

{'FB': array([2, 5], dtype=int64),
 'GOOG': array([0, 4], dtype=int64),
 'MSFT': array([1, 3], dtype=int64)}

In [209]:
by_comp.groups

{'FB': [2, 5], 'GOOG': [0, 4], 'MSFT': [1, 3]}

In [210]:
by_comp.groups["FB"]

Int64Index([2, 5], dtype='int64')

In [211]:
by_comp.get_group("FB")

,Company,Person,Sales
2,FB,Amy,340
5,FB,Sarah,350


Apply some operations/traitments 

In [212]:
by_comp.head(1)

,Company,Person,Sales
0,GOOG,Sam,200
1,MSFT,Charlie,120
2,FB,Amy,340


In [213]:
by_comp.describe()

Sales                                                       
        count   mean        std    min     25%    50%     75%    max
Company                                                             
FB        2.0  345.0   7.071068  340.0  342.50  345.0  347.50  350.0
GOOG      2.0  221.5  30.405592  200.0  210.75  221.5  232.25  243.0
MSFT      2.0  122.0   2.828427  120.0  121.00  122.0  123.00  124.0

And then call aggregate methods off the object:

In [215]:
#Only Numerical columns
by_comp.mean()

,Sales
Company,
FB,345.0
GOOG,221.5
MSFT,122.0


More examples of aggregate methods:

In [35]:
by_comp.std()

,Sales
Company,
FB,7.071068
GOOG,30.405592
MSFT,2.828427


In [11]:
by_comp.min()

,Person,Sales
Company,,
FB,Carl,243
GOOG,Charlie,120
MSFT,Amy,124


In [12]:
by_comp.max()

,Person,Sales
Company,,
FB,Sarah,350
GOOG,Sam,200
MSFT,Vanessa,340


In [216]:
#non-Na values
by_comp.count()

,Person,Sales
Company,,
FB,2,2
GOOG,2,2
MSFT,2,2


In [53]:
by_comp.count().reset_index()

,Company,Person,Sales
0,FB,2,2
1,GOOG,2,2
2,MSFT,2,2


In [236]:
by_comp.describe().loc['FB']

Sales  count      2.000000
       mean     345.000000
       std        7.071068
       min      340.000000
       25%      342.500000
       50%      345.000000
       75%      347.500000
       max      350.000000
Name: FB, dtype: float64

In [222]:
x = by_comp.describe().transpose()

In [231]:
x

Company              FB        GOOG        MSFT
Sales count    2.000000    2.000000    2.000000
      mean   345.000000  221.500000  122.000000
      std      7.071068   30.405592    2.828427
      min    340.000000  200.000000  120.000000
      25%    342.500000  210.750000  121.000000
      50%    345.000000  221.500000  122.000000
      75%    347.500000  232.250000  123.000000
      max    350.000000  243.000000  124.000000

In [232]:
x.index

MultiIndex([('Sales', 'count'),
            ('Sales',  'mean'),
            ('Sales',   'std'),
            ('Sales',   'min'),
            ('Sales',   '25%'),
            ('Sales',   '50%'),
            ('Sales',   '75%'),
            ('Sales',   'max')],
           )

In [234]:
x.columns

Index(['FB', 'GOOG', 'MSFT'], dtype='object', name='Company')

In [16]:
by_comp.describe().transpose()['GOOG']

Sales  count      2.000000
       mean     160.000000
       std       56.568542
       min      120.000000
       25%      140.000000
       50%      160.000000
       75%      180.000000
       max      200.000000
Name: GOOG, dtype: float64

## Example 1 : dataset of historical members of Congress. 

In [238]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "dataset/legislators-historical.csv",
    dtype=dtypes,
    na_values=[np.nan],
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

In [239]:
df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

In [240]:
df

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11977,Isakson,John,1944-12-28,M,sen,GA,Republican
11978,Hunter,Duncan,1976-12-07,M,rep,CA,Republican
11979,Meadows,Mark,1959-07-28,M,rep,NC,Republican
11980,Ratcliffe,John,1965-10-20,M,rep,TX,Republican


In [241]:
# Display the number of row containing a nan value
df.isna().any(axis=1)

0        False
1         True
2         True
3         True
4         True
         ...  
11977    False
11978    False
11979    False
11980    False
11981    False
Length: 11982, dtype: bool

In [242]:
# display Na row
df[df.isna().any(axis=1)]

,last_name,first_name,birthday,gender,type,state,party
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
5,Contee,Benjamin,NaT,M,rep,MD,NaN
...,...,...,...,...,...,...,...
8880,Elizalde,Joaquin,1896-08-02,M,rep,PI,NaN
9045,Romulo,Carlos,1899-01-14,M,rep,PI,NaN
9188,Rohrbough,Edward,NaT,M,rep,WV,Republican
9798,Burkhalter,Everett,NaT,M,rep,CA,Democrat


In [243]:
# Drop duplicated row 
df.drop_duplicates()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11977,Isakson,John,1944-12-28,M,sen,GA,Republican
11978,Hunter,Duncan,1976-12-07,M,rep,CA,Republican
11979,Meadows,Mark,1959-07-28,M,rep,NC,Republican
11980,Ratcliffe,John,1965-10-20,M,rep,TX,Republican


In [244]:
# Return a Series containing (Multindex)counts of unique rows in the DataFrame. it ignore rows if Nan  values
a = df.value_counts()

In [245]:
a.index

MultiIndex([( 'du Pont',     'Thomas', '1863-12-11', 'M', 'sen', 'DE', ...),
            (  'Gamble',       'John', '1848-01-15', 'M', 'rep', 'SD', ...),
            ( 'Gannett',  'Barzillai', '1764-06-17', 'M', 'rep', 'MA', ...),
            (   'Ganly',      'James', '1878-09-13', 'M', 'rep', 'NY', ...),
            (   'Gandy',      'Harry', '1881-08-13', 'M', 'rep', 'SD', ...),
            ( 'Gammage',     'Robert', '1938-03-13', 'M', 'rep', 'TX', ...),
            ('Gambrill',    'Stephen', '1873-10-02', 'M', 'rep', 'MD', ...),
            ('Gambrell',      'David', '1929-12-20', 'M', 'sen', 'GA', ...),
            (  'Gamble',     'Robert', '1851-02-07', 'M', 'sen', 'SD', ...),
            (  'Gamble',      'Ralph', '1885-05-06', 'M', 'rep', 'NY', ...),
            ...
            ('Nicholls',       'John', '1834-04-25', 'M', 'rep', 'GA', ...),
            ('Nicholas',     'Wilson', '1761-01-31', 'M', 'rep', 'VA', ...),
            ( 'Niblack',    'William', '1822-05-19', 'M', 'r

In [246]:
# The number of state 
df["state"].unique().size

58

In [247]:
# the number of member by of USA States
n_by_state = df.groupby("state")[["last_name"]].count()
n_by_state.reset_index(inplace=True)
n_by_state.sort_values(by=['last_name'], ascending=False)

,state,last_name
37,NY,1462
42,PA,1053
38,OH,674
17,IL,486
51,VA,432
22,MA,426
20,KY,373
5,CA,363
34,NJ,359
30,NC,354


In [248]:
# grouping jointly on two columns, which finds the count of Congressional members 
#broken out by state and then by gender:
b = df.groupby(["state", "gender"])["last_name"].count()
b

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
                ... 
WI     M         197
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 116, dtype: int64

In [249]:
type(b)

pandas.core.series.Series

In [250]:
b.index

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F'),
            ('AR', 'M'),
            ('AS', 'F'),
            ('AS', 'M'),
            ('AZ', 'F'),
            ('AZ', 'M'),
            ...
            ('VT', 'F'),
            ('VT', 'M'),
            ('WA', 'F'),
            ('WA', 'M'),
            ('WI', 'F'),
            ('WI', 'M'),
            ('WV', 'F'),
            ('WV', 'M'),
            ('WY', 'F'),
            ('WY', 'M')],
           names=['state', 'gender'], length=116)

In [251]:
df.index.min()

0

In [252]:
df.index.max()

11981